# 応募用ファイルの作成手順

ここでは環境構築から始めて, エージェントを作成して投稿ができるフォーマットに整理し, 実際に対戦を実行して最終的に応募用ファイルを作成するまでの流れの一例を説明する.

## 事前準備

google colaboratory上で環境構築をして実行を試みる. これによりGPU上でシミュレーションなどの実行が可能となる. まずこのファイル(`tutorial.ipynb`)を自身のドライブ(事前にgoogleアカウントを作成しておく必要がある)に保存しておき, google colaboratory上で開いておく. `seminar.zip`と配布シミュレータ`simulator_dist.zip`を`/content`直下にアップロードして, 解凍する.

In [ ]:
# osの確認
!cat /etc/os-release

In [ ]:
!unzip simulator_dist.zip
!unzip seminar.zip

`/content`直下に`simulator_dist`と`seminar`が存在していることを確認.

In [ ]:
!ls /content

## 環境構築

ローカル環境で構築する場合と同様にC++依存パッケージやその他必要なPythonライブラリをインストールしてからシミュレータ本体をインストールする.

### C++依存パッケージなどのインストール

In [ ]:
!apt-get -y install libboost-dev libnlopt-cxx-dev freeglut3-dev

In [ ]:
%cd /usr/local
!wget -nc https://gitlab.com/libeigen/eigen/-/archive/3.4.0/eigen-3.4.0.zip
!unzip eigen-3.4.0.zip
!rm eigen-3.4.0.zip
%cd eigen-3.4.0
!mkdir build
%cd build
!cmake ..
!make
!make install

In [ ]:
%cd /usr/local/include
!mkdir pybind11_json
!cp /content/simulator_dist/root/thirdParty/include/pybind11_json/pybind11_json.hpp pybind11_json/pybind11_json.hpp
!mkdir magic_enum
!wget -O magic_enum/magic_enum.hpp https://github.com/Neargye/magic_enum/releases/download/v0.9.3/magic_enum.hpp
!mkdir thread-pool
!wget -O thread-pool/BS_thread_pool.hpp https://raw.githubusercontent.com/bshoshany/thread-pool/v3.3.0/BS_thread_pool.hpp

In [ ]:
!git clone https://github.com/nlohmann/json.git -b v3.11.2 --depth 1
!cp -r json/include/nlohmann .
!rm -r json

In [ ]:
!chmod 644 pybind11_json/pybind11_json.hpp magic_enum/magic_enum.hpp thread-pool/BS_thread_pool.hpp

In [ ]:
!pip install timeout-decorator==0.5.0 keras-rl==0.4.2 pfrl==0.3.0

Pythonのバージョンは推奨バージョン(3.8)とは違うことに注意(現時点では3.10.12).

また, Pytorchの推奨バージョンは`1.13.1`なので, 厳密に合わせたい場合は改めてインストールしなおしておく.

```bash
!pip install torch==1.13.1
```

### シミュレータのインストール

In [ ]:
%cd /usr/local/src
!cp -r /content/simulator_dist/root ASRCAISim1/

In [ ]:
%cd /usr/local/src/ASRCAISim1
!pip install .

In [ ]:
%cd /usr/local/src/ASRCAISim1/sample/modules/OriginalModelSample
!pip install .

10分以上かかる.

### 実行テスト

実際に正常に動くか確認する. `readme.md`にあるようにサンプルコードを実行する.

In [ ]:
%cd /content/simulator_dist/root/sample/scripts/R5Contest/MinimumEvaluation/
!python evaluator.py "Rule-Fixed" "Rule-Fixed" -n 1 -l "./results"

 なお, GUIで再生はできない模様.

### その他

Dockerで構築する場合は容量などに余裕があればCPU版とGPU版を両方構築しておいて, 強化学習したい場合はGPU版を, 投稿テストをしたい場合はCPU版を使用するなどの方法がある.

composeファイルを以下のように編集し,

```yaml
version: "3"
services:
  dev1:
    build:
      context: .
      dockerfile: Dockerfile.cpu
    container_name: atla_cpu_gui
    ports:
      - "8081:8080"
    volumes:
      - .:/workspace
    tty: true
    user: root
  dev2:
    build:
      context: .
      dockerfile: Dockerfile.gpu
    deploy:
      resources:
        reservations:
          devices:
            - driver: nvidia
              count: 1
              capabilities: [gpu]
    container_name: atla_gpu_gui
    ports:
      - "8082:8080"
    volumes:
      - .:/workspace
    tty: true
    user: root
```

docker composeコマンドを実行する.

```bash
docker compose up -d
```

すると, CPU版のコンテナ`atla_cpu_gui`とGPU版のコンテナ`atla_gpu_gui`が構築される. 後は任意の方法(`docker exec`やVSCodeの拡張機能などの利用)でコンテナに入って作業を行う.

## エージェントの作成

基本的には`simulator_dist/make_agent.ipynb`などに従うが, observationを作成する`make_obs`メソッドやその型(観測空間)を定義する`observation_space`メソッド, 行動空間を定義する`action_space`メソッドの作成の仕方についていくつか注意点がある.

### `make_obs`メソッドと`observation_space`メソッドの実装

actionを生成するためのobservationを作成する`make_obs`メソッドを実装する.

- 強化学習を実装したい場合は基本的には`make_obs`メソッドで返る値は`numpy.ndarray`とすればよい(policyモデルに渡す想定).
  - `observation_space`メソッドで`shape`を定義する必要がある. あらかじめどのようなobservationを作成するかを設計したうえで定義する.
  - 対戦ログとしてobservationを取得するにはjson形式で保存できることが前提であるが, 今回の場合は`numpy.ndarray`は自動的に`list`に変換される.
  - その他の型を返す場合はデフォルトでjson形式で保存できるような処理を挟む必要がある(`observation_space`メソッドとの整合も考慮).
  - 特にログが欲しくない場合はそのままでよい(保存に失敗した場合, ログは勝敗結果と自分の陣営の色のみの情報になるが対戦結果自体はリーダーボードに反映される).
- 行動判断モデルとして特に機械学習モデルを使用しないアルゴリズムを実装している場合は保存しておきたい情報をjson形式で保存できるように処理したものを返せばよい.
  - そもそもログが必要ないなら適当に`0`などの値を返しておく.

観測可能な主な情報は以下の通り.

1. 自分と味方の機体諸元(位置, 速度, 姿勢, 角速度, 残弾数)
1. 自分と味方の誘導弾諸元(位置, 速度, 目標ID, 誘導状態)
1. 相手の機体諸元(位置と速度のみ)
1. 相手の誘導弾諸元(方向のみ)
1. 戦闘開始からの経過時間

詳細は`docs/基準モデル及びパラメータに関する資料.pdf`の1.4.6項を参照されたい. 観測情報は階層構造になっていることに注目して例えば彼機の航跡は以下のような形で取得する.

```Python
parent.observables.at_p("/sensor/track")
```

以下では例として数値ベクトル(自機の位置ベクトルと速度ベクトル, 検知している彼機の位置ベクトルと速度ベクトル, 自機の誘導弾の位置ベクトルと速度ベクトル, 残誘導弾の割合とmwsで検知している2次元航跡データを単純に並べてつなげた1次元の配列)として生成する. observationはステップごとにログとして残すことが可能であるが, json形式で保存できるフォーマットである必要があるが, 前述の通り`numpy.ndarray`として定義している場合は特に気にすることはない. それ以外の場合はあらかじめPythonプリミティブの型として作成するフラグ(ここでは`to_list`とする. 例えばconfigファイルで定義しておいて, インスタンス化したときに定義できるようにしておく)を定義しておいて, 適宜切り替えができるようにしておくとよい.

`OriginalModelSample`モジュールでは単純な数値ベクトルだけでなく画像データとして生成するなど, より細かい作成方法が実装されているので, `R5PyAgentSample01S.py`や`R5PyAgentSample01M.py`などを参照されたい.

In [ ]:
import numpy as np

def makeObs(self):
    # 味方機(自機含む)
    self.ourMotion=[]
    self.ourObservables=[]

    for pIdx,parent in enumerate(self.parents.values()):
        if(parent.isAlive()):
            firstAlive=parent
            break
    for pIdx,parent in enumerate(self.parents.values()):
        if(parent.isAlive()):
            #残存していればobservablesそのもの
            self.ourMotion.append(parent.observables["motion"]())
            self.ourObservables.append(parent.observables)
            myMotion=MotionState(parent.observables["motion"])

        else:
            self.ourMotion.append({})
            #被撃墜or墜落済なら本体の更新は止まっているので残存している親が代理更新したものを取得(誘導弾情報のため)
            self.ourObservables.append(
                firstAlive.observables.at_p("/shared/fighter").at(parent.getFullName()))

    # 彼機(味方の誰かが探知しているもののみ)
    # 観測されている航跡を、自陣営の機体に近いものから順にソートしてlastTrackInfoに格納する。
    # lastTrackInfoは行動のdeployでも射撃対象の指定のために参照する。
    def distance(track):
        ret=-1.0
        for pIdx,parent in enumerate(self.parents.values()):
            if(parent.isAlive()):
                myMotion=MotionState(parent.observables["motion"])
                tmp=np.linalg.norm(track.posI()-myMotion.pos)
                if(ret<0 or tmp<ret):
                    ret=tmp
        return ret
    for pIdx,parent in enumerate(self.parents.values()):
        if(parent.isAlive()):
            self.lastTrackInfo=sorted([Track3D(t) for t in parent.observables.at_p("/sensor/track")],key=distance) # type: ignore
            break

    # 味方誘導弾(射撃時刻が古いものから最大N発分)
    # 味方の誘導弾を射撃時刻の古い順にソート
    def launchedT(m):
        return m["launchedT"]() if m["isAlive"]() and m["hasLaunched"]() else np.inf
    self.msls=sorted(sum([[m for m in f.at_p("/weapon/missiles")] for f in self.ourObservables],[]),key=launchedT)

    f_vec = [0.0]*6
    for fIdx in range(len(self.ourMotion)):
        if(fIdx>=self.maxTrackNum["Friend"]):
            break

        if(self.ourObservables[fIdx]["isAlive"]()):
            #初期弾数
            numMsls=self.ourObservables[fIdx].at_p("/spec/weapon/numMsls")()
            #残弾数
            remMsls=self.ourObservables[fIdx].at_p("/weapon/remMsls")()

            f_vec[3*fIdx] = remMsls/numMsls
            #MWS検出情報
            def angle(track):
                return -np.dot(track.dirI(),myMotion.relBtoP(np.array([1,0,0])))
            mws=sorted([Track2D(t) for t in self.ourObservables[fIdx].at_p("/sensor/mws/track")],key=angle)
            if len(mws):
                f_vec[3*fIdx+1] = mws[0].dirI()[0]
                f_vec[3*fIdx+2] = mws[0].dirI()[1]

    # observationの作成
    om_vec = []
    for om in self.ourMotion:
        if len(om)==0:
            om_vec += [0.0]*6
        else:
            om_vec += om['pos']+om['vel']

    lt_vec = []
    for lt in self.lastTrackInfo:
        lt_vec += list(lt.pos)+list(lt.vel)
    n_lt = len(lt_vec)
    if n_lt < 12:
        lt_vec += [0.0]*(12-n_lt)

    msls = [m() for m in self.msls]
    m_vec = []
    for msl in msls:
        if msl['isAlive']:
            m_vec += msl['motion']['pos']+msl['motion']['vel']
        else:
            m_vec += [0.0]*6

    vec = om_vec + lt_vec + m_vec + f_vec

    return np.array(vec, dtype=np.float32)

observationの形を定義する`observation_space`メソッドを実装する. observationは78次元の実数値ベクトルとなるのでそのように定義する.

In [ ]:
import numpy as np
from gymnasium import spaces

def observation_space(self):
    return spaces.Box(low=-np.inf,high=np.inf,shape=(78,))

### `action_space`メソッドの実装

actionは`deploy`メソッドに渡されて自機の動きや誘導弾発射の意思決定などを行うためのもの.

以下の項目に関連するような行動を定義しておく.

1. 自分と味方の機動（以下のいずれかによる）
    - ロール、ピッチ、ヨー、スロットルの直接出力
    - 進みたい方向と進みたい速度による抽象的な出力
1. 射撃有無と射撃対象

actionが存在すべき行動空間(このあと実装するpolicyの出力するactionの種類数)を定義する. ここではインスタンス化したときに各種設定(`self.actionDims`などの形でインスタンス化したときにshapeを定義しておく)をして(`OriginalModelSample/R5PyAgentSample01M.py`からの抜粋)10種類の離散値とする.

なお, 強化学習を前提としない場合は特に気にすることはなく何らかの適当な値を返すような実装としておいてもよい(`spaces.Discrete()`を返すなど).

In [ ]:
from gymnasium import spaces

def action_space(self):
    return spaces.MultiDiscrete(self.actionDims) # 行動空間の定義

以下は`deploy`メソッドの実装例. policyが返した`action`をどのように渡しているか確認. 

In [ ]:
def deploy(self,action):
    self.last_action_obs=np.zeros([self.maxTrackNum["Friend"],self.last_action_dim],dtype=np.float32)
    action_idx=0

    for pIdx,parent in enumerate(self.parents.values()):
        if(not parent.isAlive()):
            continue
        actionInfo=self.actionInfos[pIdx]
        myMotion=MotionState(parent.observables["motion"])
        #左右旋回
        deltaAz=self.turnTable[action[action_idx]]
        def angle(track):
            return -np.dot(track.dirI(),myMotion.relBtoP(np.array([1,0,0])))
        mws=sorted([Track2D(t) for t in parent.observables.at_p("/sensor/mws/track")],key=angle)
        if(len(mws)>0 and self.use_override_evasion):
            deltaAz=self.evasion_turnTable[action[action_idx]]
            dr=np.zeros([3])
            for m in mws:
                dr+=m.dirI()
            dr/=np.linalg.norm(dr)
            dstAz=atan2(-dr[1],-dr[0])+deltaAz
            actionInfo.dstDir=np.array([cos(dstAz),sin(dstAz),0])
        elif(self.dstAz_relative):
            actionInfo.dstDir=myMotion.relHtoP(np.array([cos(deltaAz),sin(deltaAz),0]))
        else:
            actionInfo.dstDir=self.teamOrigin.relBtoP(np.array([cos(deltaAz),sin(deltaAz),0]))
        action_idx+=1
        dstAz=atan2(actionInfo.dstDir[1],actionInfo.dstDir[0])
        self.last_action_obs[pIdx,0]=dstAz

        #上昇・下降
        if(self.use_altitude_command):
            refAlt=round(-myMotion.pos(2)/self.refAltInterval)*self.refAltInterval
            actionInfo.dstAlt=max(self.altMin,min(self.altMax,refAlt+self.altTable[action[action_idx]]))
            dstPitch=0#dstAltをcommandsに与えればSixDoFFighter::FlightControllerのaltitudeKeeperで別途計算されるので0でよい。
        else:
            dstPitch=self.pitchTable[action[action_idx]]
        action_idx+=1
        actionInfo.dstDir=np.array([actionInfo.dstDir[0]*cos(dstPitch),actionInfo.dstDir[1]*cos(dstPitch),-sin(dstPitch)])
        self.last_action_obs[pIdx,1]=actionInfo.dstAlt if self.use_altitude_command else dstPitch

        #加減速
        V=np.linalg.norm(myMotion.vel)
        if(self.always_maxAB):
            actionInfo.asThrottle=True
            actionInfo.keepVel=False
            actionInfo.dstThrottle=1.0
            self.last_action_obs[pIdx,2]=1.0
        else:
            actionInfo.asThrottle=False
            accel=self.accelTable[action[action_idx]]
            action_idx+=1
            actionInfo.dstV=V+accel # type: ignore
            actionInfo.keepVel = accel==0.0
            self.last_action_obs[pIdx,2]=accel/max(self.accelTable[-1],self.accelTable[0])
        #下限速度の制限
        if(V<self.minimumV):
            actionInfo.velRecovery=True
        if(V>=self.minimumRecoveryV):
            actionInfo.velRecovery=False
        if(actionInfo.velRecovery):
            actionInfo.dstV=self.minimumRecoveryDstV
            actionInfo.asThrottle=False

        #射撃
        #actionのパース
        shotTarget=action[action_idx]-1
        action_idx+=1
        if(self.use_Rmax_fire):
            if(len(self.shotIntervalTable)>1):
                shotInterval=self.shotIntervalTable[action[action_idx]]
                action_idx+=1
            else:
                shotInterval=self.shotIntervalTable[0]
            if(len(self.shotThresholdTable)>1):
                shotThreshold=self.shotThresholdTable[action[action_idx]]
                action_idx+=1
            else:
                shotThreshold=self.shotThresholdTable[0]
        #射撃可否の判断、射撃コマンドの生成
        flyingMsls=0
        for msl in parent.observables.at_p("/weapon/missiles"):
            if(msl.at("isAlive")() and msl.at("hasLaunched")()):
                flyingMsls+=1
        if(
                shotTarget>=0 and
                shotTarget<len(self.lastTrackInfo) and
                parent.isLaunchableAt(self.lastTrackInfo[shotTarget]) and
                flyingMsls<self.maxSimulShot
        ):
                if(self.use_Rmax_fire):
                rMin=np.inf
                t=self.lastTrackInfo[shotTarget]
                r=self.calcRNorm(parent,myMotion,t)
                if(r<=shotThreshold):
                    #射程の条件を満たしている
                    if(not t.truth in actionInfo.lastShotTimes):
                        actionInfo.lastShotTimes[t.truth]=0.0
                    if(self.manager.getTime()-actionInfo.lastShotTimes[t.truth]>=shotInterval):
                        #射撃間隔の条件を満たしている
                        actionInfo.lastShotTimes[t.truth]=self.manager.getTime()
                    else:
                        #射撃間隔の条件を満たさない
                        shotTarget=-1
                else:
                    #射程の条件を満たさない
                    shotTarget=-1
        else:
            shotTarget=-1
        self.last_action_obs[pIdx,3+(shotTarget+1)]=1
        if(shotTarget>=0):
            actionInfo.launchFlag=True
            actionInfo.target=self.lastTrackInfo[shotTarget]
        else:
            actionInfo.launchFlag=False
            actionInfo.target=Track3D()
        
        self.observables[parent.getFullName()]["decision"]={
            "Roll":("Don't care"),
            "Fire":(actionInfo.launchFlag,actionInfo.target.to_json())
        }
        if(len(mws)>0 and self.use_override_evasion):
            self.observables[parent.getFullName()]["decision"]["Horizontal"]=("Az_NED",dstAz)
        else:
            if(self.dstAz_relative):
                self.observables[parent.getFullName()]["decision"]["Horizontal"]=("Az_BODY",deltaAz)
            else:
                self.observables[parent.getFullName()]["decision"]["Horizontal"]=("Az_NED",dstAz)
        if(self.use_altitude_command):
            self.observables[parent.getFullName()]["decision"]["Vertical"]=("Pos",-actionInfo.dstAlt)
        else:
            self.observables[parent.getFullName()]["decision"]["Vertical"]=("El",-dstPitch)
        if(actionInfo.asThrottle):
            self.observables[parent.getFullName()]["decision"]["Throttle"]=("Throttle",actionInfo.dstThrottle)
        else:
            self.observables[parent.getFullName()]["decision"]["Throttle"]=("Vel",actionInfo.dstV)

### 全てをまとめる

以下では定義したactionなどに合わせて`deploy`や`control`メソッドなどを含めたすべてを示してある. `control`メソッドでは場外に出ないような制御をかけている. これを`MyAgent.py`として作成しておく(中央集権方式).

In [ ]:
import numpy as np
from gymnasium import spaces
from math import cos, sin, atan2, sqrt
from ASRCAISim1.libCore import Agent, MotionState, Track3D, Track2D, getValueFromJsonKRD, deg2rad, StaticCollisionAvoider2D, LinearSegment, AltitudeKeeper


class MyAgent(Agent):

    class TeamOrigin():
        #陣営座標系(進行方向が+x方向となるようにz軸まわりに回転させ、防衛ライン中央が原点となるように平行移動させた座標系)を表すクラス。
        #MotionStateを使用しても良いがクォータニオンを経由することで浮動小数点演算に起因する余分な誤差が生じるため、もし可能な限り対称性を求めるのであればこの例のように符号反転で済ませたほうが良い。
        #ただし、機体運動等も含めると全ての状態量に対して厳密に対称なシミュレーションとはならないため、ある程度の誤差は生じる。
        def __init__(self,isEastSider_,dLine):
            self.isEastSider=isEastSider_
            if(self.isEastSider):
                self.pos=np.array([0.,dLine,0.])
            else:
                self.pos=np.array([0.,-dLine,0.])
        def relBtoP(self,v):
            #陣営座標系⇛慣性座標系
            if(self.isEastSider):
                return np.array([v[1],-v[0],v[2]])
            else:
                return np.array([-v[1],v[0],v[2]])
        def relPtoB(self,v):
            #慣性座標系⇛陣営座標系
            if(self.isEastSider):
                return np.array([-v[1],v[0],v[2]])
            else:
                return np.array([v[1],-v[0],v[2]])


    class ActionInfo():
        def __init__(self):
            self.dstDir=np.array([1.0,0.0,0.0]) #目標進行方向
            self.dstAlt=10000.0 #目標高度
            self.velRecovery=False #下限速度制限からの回復中かどうか
            self.asThrottle=False #加減速についてスロットルでコマンドを生成するかどうか
            self.keepVel=False #加減速について等速(dstAccel=0)としてコマンドを生成するかどうか
            self.dstThrottle=1.0 #目標スロットル
            self.dstV=300.0 #目標速度
            self.launchFlag=False #射撃するかどうか
            self.target=Track3D() #射撃対象
            self.lastShotTimes={} #各Trackに対する直前の射撃時刻


    def __init__(self,modelConfig,instanceConfig):
        super().__init__(modelConfig,instanceConfig) # 設定の読み込み
        if(self.isDummy):
            return # Factoryによるダミー生成のために空引数でのインスタンス化に対応させる
        self.time_params = self.modelConfig['interval']()
        self.to_list = self.modelConfig['to_list']()
        self.own = self.getTeam()
        self.maxTrackNum=getValueFromJsonKRD(self.modelConfig,"maxTrackNum",self.randomGen,{"Friend":4,"Enemy":4})
        self.last_action_dim=3+(1+self.maxTrackNum["Enemy"])
        self.maxMissileNum=getValueFromJsonKRD(self.modelConfig,"maxMissileNum",self.randomGen,{"Friend":8,"Enemy":1})
        # 場外制限に関する設定
        self.dOutLimit=getValueFromJsonKRD(self.modelConfig,"dOutLimit",self.randomGen,5000.0)
        self.dOutLimitThreshold=getValueFromJsonKRD(self.modelConfig,"dOutLimitThreshold",self.randomGen,10000.0)
        self.dOutLimitStrength=getValueFromJsonKRD(self.modelConfig,"dOutLimitStrength",self.randomGen,2e-3)
        
        # 左右旋回に関する設定
        self.turnTable=np.array(sorted(getValueFromJsonKRD(self.modelConfig,"turnTable",self.randomGen,
			[-90.0,-45.0,-20.0,-10.0,0.0,10.0,20.0,45.0,90.0])),dtype=np.float64)
        self.turnTable*=deg2rad(1.0)
        self.use_override_evasion=getValueFromJsonKRD(self.modelConfig,"use_override_evasion",self.randomGen,True)
        if(self.use_override_evasion):
            self.evasion_turnTable=np.array(sorted(getValueFromJsonKRD(self.modelConfig,"evasion_turnTable",self.randomGen,
                    [-90.0,-45.0,-20.0,-10.0,0.0,10.0,20.0,45.0,90.0])),dtype=np.float64)
            self.evasion_turnTable*=deg2rad(1.0)
            assert(len(self.turnTable)==len(self.evasion_turnTable))
        else:
            self.evasion_turnTable=self.turnTable
        self.dstAz_relative=getValueFromJsonKRD(self.modelConfig,"dstAz_relative",self.randomGen,False)

		# 上昇・下降に関する設定
        self.use_altitude_command=getValueFromJsonKRD(self.modelConfig,"use_altitude_command",self.randomGen,False)
        if(self.use_altitude_command):
            self.altTable=np.array(sorted(getValueFromJsonKRD(self.modelConfig,"altTable",self.randomGen,
				[-8000.0,-4000.0,-2000.0,-1000.0,0.0,1000.0,2000.0,4000.0,8000.0])),dtype=np.float64)
            self.refAltInterval=getValueFromJsonKRD(self.modelConfig,"refAltInterval",self.randomGen,1000.0)
        else:
            self.pitchTable=np.array(sorted(getValueFromJsonKRD(self.modelConfig,"pitchTable",self.randomGen,
				[-45.0,-20.0,-10.0,-5.0,0.0,5.0,10.0,20.0,45.0])),dtype=np.float64)
            self.pitchTable*=deg2rad(1.0)
            self.refAltInterval=1.0
        
        # 加減速に関する設定
        self.accelTable=np.array(sorted(getValueFromJsonKRD(self.modelConfig,"accelTable",self.randomGen,[-2.0,0.0,2.0])),dtype=np.float64)
        self.always_maxAB=getValueFromJsonKRD(self.modelConfig,"always_maxAB",self.randomGen,False)
        
        # 射撃に関する設定
        self.use_Rmax_fire=getValueFromJsonKRD(self.modelConfig,"use_Rmax_fire",self.randomGen,False)
        if(self.use_Rmax_fire):
            self.shotIntervalTable=np.array(sorted(getValueFromJsonKRD(self.modelConfig,"shotIntervalTable",self.randomGen,
				[5.0,10.0,20.0,40.0,80.0])),dtype=np.float64)
            self.shotThresholdTable=np.array(sorted(getValueFromJsonKRD(self.modelConfig,"shotThresholdTable",self.randomGen,
				[0.0,0.25,0.5,0.75,1.0])),dtype=np.float64)
        #行動制限に関する設定
        #  高度制限に関する設定
        self.altMin=getValueFromJsonKRD(self.modelConfig,"altMin",self.randomGen,2000.0)
        self.altMax=getValueFromJsonKRD(self.modelConfig,"altMax",self.randomGen,15000.0)
        self.altitudeKeeper=AltitudeKeeper(modelConfig().get("altitudeKeeper",{}))
        # 同時射撃数の制限に関する設定
        self.maxSimulShot=getValueFromJsonKRD(self.modelConfig,"maxSimulShot",self.randomGen,4)
        # 下限速度の制限に関する設定
        self.minimumV=getValueFromJsonKRD(self.modelConfig,"minimumV",self.randomGen,150.0)
        self.minimumRecoveryV=getValueFromJsonKRD(self.modelConfig,"minimumRecoveryV",self.randomGen,180.0)
        self.minimumRecoveryDstV=getValueFromJsonKRD(self.modelConfig,"minimumRecoveryDstV",self.randomGen,200.0)
        nvec=[]
        for pIdx,parent in enumerate(self.parents.values()):
            nvec.append(len(self.turnTable))
            nvec.append(len(self.altTable) if self.use_altitude_command else len(self.pitchTable))
            if(not self.always_maxAB):
                nvec.append(len(self.accelTable))
            nvec.append(self.maxTrackNum["Enemy"]+1)
            if(self.use_Rmax_fire):
                if(len(self.shotIntervalTable)>1):
                    nvec.append(len(self.shotIntervalTable))
                if(len(self.shotThresholdTable)>1):
                    nvec.append(len(self.shotThresholdTable))
        self.totalActionDim=1
        self.actionDims=np.zeros([len(nvec)], dtype=int)
        for i in range(len(nvec)):
            self.totalActionDim*=nvec[i]
            self.actionDims[i]=nvec[i]

        self.actionInfos=[self.ActionInfo() for _ in self.parents]


    def observation_space(self):
        return spaces.Box(low=-np.inf, high=np.inf, shape=(78,))


    def action_space(self):
        return spaces.MultiDiscrete(self.actionDims)


    def validate(self):
        #Rulerに関する情報の取得
        rulerObs=self.manager.getRuler()().observables()
        self.dOut=rulerObs["dOut"] # 戦域中心から場外ラインまでの距離
        self.dLine=rulerObs["dLine"] # 戦域中心から防衛ラインまでの距離
        self.teamOrigin=self.TeamOrigin(self.own==rulerObs["eastSider"],self.dLine) # 陣営座標系変換クラス定義


    def makeObs(self):
        # 味方機(自機含む)
        self.ourMotion=[]
        self.ourObservables=[]

        for pIdx,parent in enumerate(self.parents.values()):
            if(parent.isAlive()):
                firstAlive=parent
                break
        for pIdx,parent in enumerate(self.parents.values()):
            if(parent.isAlive()):
                #残存していればobservablesそのもの
                self.ourMotion.append(parent.observables["motion"]())
                self.ourObservables.append(parent.observables)
                myMotion=MotionState(parent.observables["motion"])

            else:
                self.ourMotion.append({})
                #被撃墜or墜落済なら本体の更新は止まっているので残存している親が代理更新したものを取得(誘導弾情報のため)
                self.ourObservables.append(
                    firstAlive.observables.at_p("/shared/fighter").at(parent.getFullName()))

        # 彼機(味方の誰かが探知しているもののみ)
        # 観測されている航跡を、自陣営の機体に近いものから順にソートしてlastTrackInfoに格納する。
        # lastTrackInfoは行動のdeployでも射撃対象の指定のために参照する。
        def distance(track):
            ret=-1.0
            for pIdx,parent in enumerate(self.parents.values()):
                if(parent.isAlive()):
                    myMotion=MotionState(parent.observables["motion"])
                    tmp=np.linalg.norm(track.posI()-myMotion.pos)
                    if(ret<0 or tmp<ret):
                        ret=tmp
            return ret
        for pIdx,parent in enumerate(self.parents.values()):
            if(parent.isAlive()):
                self.lastTrackInfo=sorted([Track3D(t) for t in parent.observables.at_p("/sensor/track")],key=distance) # type: ignore
                break

        # 味方誘導弾(射撃時刻が古いものから最大N発分)
        # 味方の誘導弾を射撃時刻の古い順にソート
        def launchedT(m):
            return m["launchedT"]() if m["isAlive"]() and m["hasLaunched"]() else np.inf
        self.msls=sorted(sum([[m for m in f.at_p("/weapon/missiles")] for f in self.ourObservables],[]),key=launchedT)

        f_vec = [0.0]*6
        for fIdx in range(len(self.ourMotion)):
            if(fIdx>=self.maxTrackNum["Friend"]):
                break

            if(self.ourObservables[fIdx]["isAlive"]()):
                #初期弾数
                numMsls=self.ourObservables[fIdx].at_p("/spec/weapon/numMsls")()
                #残弾数
                remMsls=self.ourObservables[fIdx].at_p("/weapon/remMsls")()

                f_vec[3*fIdx] = remMsls/numMsls
                #MWS検出情報
                def angle(track):
                    return -np.dot(track.dirI(),myMotion.relBtoP(np.array([1,0,0])))
                mws=sorted([Track2D(t) for t in self.ourObservables[fIdx].at_p("/sensor/mws/track")],key=angle)
                if len(mws):
                    f_vec[3*fIdx+1] = mws[0].dirI()[0]
                    f_vec[3*fIdx+2] = mws[0].dirI()[1]

        # observationの作成
        om_vec = []
        for om in self.ourMotion:
            if len(om)==0:
                om_vec += [0.0]*6
            else:
                om_vec += om['pos']+om['vel']

        lt_vec = []
        for lt in self.lastTrackInfo:
            lt_vec += list(lt.pos)+list(lt.vel)
        n_lt = len(lt_vec)
        if n_lt < 12:
            lt_vec += [0.0]*(12-n_lt)

        msls = [m() for m in self.msls]
        m_vec = []
        for msl in msls:
            if msl['isAlive']:
                m_vec += msl['motion']['pos']+msl['motion']['vel']
            else:
                m_vec += [0.0]*6

        vec = om_vec + lt_vec + m_vec + f_vec

        return np.array(vec, dtype=np.float32)


    def deploy(self,action):
        self.last_action_obs=np.zeros([self.maxTrackNum["Friend"],self.last_action_dim],dtype=np.float32)
        action_idx=0

        for pIdx,parent in enumerate(self.parents.values()):
            if(not parent.isAlive()):
                continue
            actionInfo=self.actionInfos[pIdx]
            myMotion=MotionState(parent.observables["motion"])
			#左右旋回
            deltaAz=self.turnTable[action[action_idx]]
            def angle(track):
                return -np.dot(track.dirI(),myMotion.relBtoP(np.array([1,0,0])))
            mws=sorted([Track2D(t) for t in parent.observables.at_p("/sensor/mws/track")],key=angle)
            if(len(mws)>0 and self.use_override_evasion):
                deltaAz=self.evasion_turnTable[action[action_idx]]
                dr=np.zeros([3])
                for m in mws:
                    dr+=m.dirI()
                dr/=np.linalg.norm(dr)
                dstAz=atan2(-dr[1],-dr[0])+deltaAz
                actionInfo.dstDir=np.array([cos(dstAz),sin(dstAz),0])
            elif(self.dstAz_relative):
                actionInfo.dstDir=myMotion.relHtoP(np.array([cos(deltaAz),sin(deltaAz),0]))
            else:
                actionInfo.dstDir=self.teamOrigin.relBtoP(np.array([cos(deltaAz),sin(deltaAz),0]))
            action_idx+=1
            dstAz=atan2(actionInfo.dstDir[1],actionInfo.dstDir[0])
            self.last_action_obs[pIdx,0]=dstAz

			#上昇・下降
            if(self.use_altitude_command):
                refAlt=round(-myMotion.pos(2)/self.refAltInterval)*self.refAltInterval
                actionInfo.dstAlt=max(self.altMin,min(self.altMax,refAlt+self.altTable[action[action_idx]]))
                dstPitch=0#dstAltをcommandsに与えればSixDoFFighter::FlightControllerのaltitudeKeeperで別途計算されるので0でよい。
            else:
                dstPitch=self.pitchTable[action[action_idx]]
            action_idx+=1
            actionInfo.dstDir=np.array([actionInfo.dstDir[0]*cos(dstPitch),actionInfo.dstDir[1]*cos(dstPitch),-sin(dstPitch)])
            self.last_action_obs[pIdx,1]=actionInfo.dstAlt if self.use_altitude_command else dstPitch

			#加減速
            V=np.linalg.norm(myMotion.vel)
            if(self.always_maxAB):
                actionInfo.asThrottle=True
                actionInfo.keepVel=False
                actionInfo.dstThrottle=1.0
                self.last_action_obs[pIdx,2]=1.0
            else:
                actionInfo.asThrottle=False
                accel=self.accelTable[action[action_idx]]
                action_idx+=1
                actionInfo.dstV=V+accel # type: ignore
                actionInfo.keepVel = accel==0.0
                self.last_action_obs[pIdx,2]=accel/max(self.accelTable[-1],self.accelTable[0])
            #下限速度の制限
            if(V<self.minimumV):
                actionInfo.velRecovery=True
            if(V>=self.minimumRecoveryV):
                actionInfo.velRecovery=False
            if(actionInfo.velRecovery):
                actionInfo.dstV=self.minimumRecoveryDstV
                actionInfo.asThrottle=False

            #射撃
            #actionのパース
            shotTarget=action[action_idx]-1
            action_idx+=1
            if(self.use_Rmax_fire):
                if(len(self.shotIntervalTable)>1):
                    shotInterval=self.shotIntervalTable[action[action_idx]]
                    action_idx+=1
                else:
                    shotInterval=self.shotIntervalTable[0]
                if(len(self.shotThresholdTable)>1):
                    shotThreshold=self.shotThresholdTable[action[action_idx]]
                    action_idx+=1
                else:
                    shotThreshold=self.shotThresholdTable[0]
            #射撃可否の判断、射撃コマンドの生成
            flyingMsls=0
            for msl in parent.observables.at_p("/weapon/missiles"):
                if(msl.at("isAlive")() and msl.at("hasLaunched")()):
                    flyingMsls+=1
            if(
                 shotTarget>=0 and
                 shotTarget<len(self.lastTrackInfo) and
                 parent.isLaunchableAt(self.lastTrackInfo[shotTarget]) and
                 flyingMsls<self.maxSimulShot
            ):
                 if(self.use_Rmax_fire):
                    rMin=np.inf
                    t=self.lastTrackInfo[shotTarget]
                    r=self.calcRNorm(parent,myMotion,t)
                    if(r<=shotThreshold):
                        #射程の条件を満たしている
                        if(not t.truth in actionInfo.lastShotTimes):
                            actionInfo.lastShotTimes[t.truth]=0.0
                        if(self.manager.getTime()-actionInfo.lastShotTimes[t.truth]>=shotInterval):
                            #射撃間隔の条件を満たしている
                            actionInfo.lastShotTimes[t.truth]=self.manager.getTime()
                        else:
                            #射撃間隔の条件を満たさない
                            shotTarget=-1
                    else:
                        #射程の条件を満たさない
                        shotTarget=-1
            else:
                shotTarget=-1
            self.last_action_obs[pIdx,3+(shotTarget+1)]=1
            if(shotTarget>=0):
                actionInfo.launchFlag=True
                actionInfo.target=self.lastTrackInfo[shotTarget]
            else:
                actionInfo.launchFlag=False
                actionInfo.target=Track3D()
            
            self.observables[parent.getFullName()]["decision"]={
                "Roll":("Don't care"),
                "Fire":(actionInfo.launchFlag,actionInfo.target.to_json())
            }
            if(len(mws)>0 and self.use_override_evasion):
                self.observables[parent.getFullName()]["decision"]["Horizontal"]=("Az_NED",dstAz)
            else:
                if(self.dstAz_relative):
                    self.observables[parent.getFullName()]["decision"]["Horizontal"]=("Az_BODY",deltaAz)
                else:
                    self.observables[parent.getFullName()]["decision"]["Horizontal"]=("Az_NED",dstAz)
            if(self.use_altitude_command):
                self.observables[parent.getFullName()]["decision"]["Vertical"]=("Pos",-actionInfo.dstAlt)
            else:
                self.observables[parent.getFullName()]["decision"]["Vertical"]=("El",-dstPitch)
            if(actionInfo.asThrottle):
                self.observables[parent.getFullName()]["decision"]["Throttle"]=("Throttle",actionInfo.dstThrottle)
            else:
                self.observables[parent.getFullName()]["decision"]["Throttle"]=("Vel",actionInfo.dstV)


    def control(self):
		#Setup collision avoider
        avoider=StaticCollisionAvoider2D()
        #北側
        c={
            "p1":np.array([+self.dOut,-5*self.dLine,0]),
            "p2":np.array([+self.dOut,+5*self.dLine,0]),
            "infinite_p1":True,
            "infinite_p2":True,
            "isOneSide":True,
            "inner":np.array([0.0,0.0]),
            "limit":self.dOutLimit,
            "threshold":self.dOutLimitThreshold,
            "adjustStrength":self.dOutLimitStrength,
        }
        avoider.borders.append(LinearSegment(c))
        #南側
        c={
            "p1":np.array([-self.dOut,-5*self.dLine,0]),
            "p2":np.array([-self.dOut,+5*self.dLine,0]),
            "infinite_p1":True,
            "infinite_p2":True,
            "isOneSide":True,
            "inner":np.array([0.0,0.0]),
            "limit":self.dOutLimit,
            "threshold":self.dOutLimitThreshold,
            "adjustStrength":self.dOutLimitStrength,
        }
        avoider.borders.append(LinearSegment(c))
        #東側
        c={
            "p1":np.array([-5*self.dOut,+self.dLine,0]),
            "p2":np.array([+5*self.dOut,+self.dLine,0]),
            "infinite_p1":True,
            "infinite_p2":True,
            "isOneSide":True,
            "inner":np.array([0.0,0.0]),
            "limit":self.dOutLimit,
            "threshold":self.dOutLimitThreshold,
            "adjustStrength":self.dOutLimitStrength,
        }
        avoider.borders.append(LinearSegment(c))
        #西側
        c={
            "p1":np.array([-5*self.dOut,-self.dLine,0]),
            "p2":np.array([+5*self.dOut,-self.dLine,0]),
            "infinite_p1":True,
            "infinite_p2":True,
            "isOneSide":True,
            "inner":np.array([0.0,0.0]),
            "limit":self.dOutLimit,
            "threshold":self.dOutLimitThreshold,
            "adjustStrength":self.dOutLimitStrength,
        }
        avoider.borders.append(LinearSegment(c))
        for pIdx,parent in enumerate(self.parents.values()):
            if(not parent.isAlive()):
                continue
            actionInfo=self.actionInfos[pIdx]
            myMotion=MotionState(parent.observables["motion"])
            pos=myMotion.pos
            vel=myMotion.vel
            #戦域逸脱を避けるための方位補正
            actionInfo.dstDir=avoider(myMotion,actionInfo.dstDir)
            #高度方向の補正(actionがピッチ指定の場合)
            if(not self.use_altitude_command):
                n=sqrt(actionInfo.dstDir[0]*actionInfo.dstDir[0]+actionInfo.dstDir[1]*actionInfo.dstDir[1])
                dstPitch=atan2(-actionInfo.dstDir[2],n)
                #高度下限側
                bottom=self.altitudeKeeper(myMotion,actionInfo.dstDir,self.altMin)
                minPitch=atan2(-bottom[2],sqrt(bottom[0]*bottom[0]+bottom[1]*bottom[1]))
                #高度上限側
                top=self.altitudeKeeper(myMotion,actionInfo.dstDir,self.altMax)
                maxPitch=atan2(-top[2],sqrt(top[0]*top[0]+top[1]*top[1]))
                dstPitch=max(minPitch,min(maxPitch,dstPitch))
                cs=cos(dstPitch)
                sn=sin(dstPitch)
                actionInfo.dstDir=np.array([actionInfo.dstDir[0]/n*cs,actionInfo.dstDir[1]/n*cs,-sn])
            self.commands[parent.getFullName()]={
                "motion":{
                    "dstDir":actionInfo.dstDir
                },
                "weapon":{
                    "launch":actionInfo.launchFlag,
                    "target":actionInfo.target.to_json()
                }
            }
            if(self.use_altitude_command):
                self.commands[parent.getFullName()]["motion"]["dstAlt"]=actionInfo.dstAlt
            if(actionInfo.asThrottle):
                self.commands[parent.getFullName()]["motion"]["dstThrottle"]=actionInfo.dstThrottle
            elif(actionInfo.keepVel):
                self.commands[parent.getFullName()]["motion"]["dstAccel"]=0.0
            else:
                self.commands[parent.getFullName()]["motion"]["dstV"]=actionInfo.dstV
            actionInfo.launchFlag=False

    
    def calcRHead(self,parent,myMotion,track):
        #相手が現在の位置、速度で直ちに正面を向いて水平飛行になった場合の射程(RHead)を返す。
        rt=track.posI()
        vt=track.velI()
        rs=myMotion.pos
        vs=myMotion.vel
        return parent.getRmax(rs,vs,rt,vt,np.pi)
    
    
    def calcRTail(self,parent,myMotion,track):
        #相手が現在の位置、速度で直ちに背後を向いて水平飛行になった場合の射程(RTail)を返す。
        rt=track.posI()
        vt=track.velI()
        rs=myMotion.pos
        vs=myMotion.vel
        return parent.getRmax(rs,vs,rt,vt,0.0)

    
    def calcRNorm(self,parent,myMotion,track):
        #RTail→0、RHead→1として正規化した距離を返す。
        RHead=self.calcRHead(parent,myMotion,track)
        RTail=self.calcRTail(parent,myMotion,track)
        rs=myMotion.pos
        rt=track.posI()
        r=np.linalg.norm(rs-rt)-RTail
        delta=RHead-RTail
        outRangeScale=100000.0
        if(delta==0):
            if(r<0):
                r=r/outRangeScale
            elif(r>0):
                r=1+r/outRangeScale
            else:
                r=0
        else:
            if(r<0):
                r=r/outRangeScale
            elif(r>delta):
                r=1+(r-delta)/outRangeScale
            else:
                r/=delta
        return r


## policyの実装

配布した`make_agent_rl/make_agent.ipynb`に従う.

observationを渡してactionを返すpolicyモデルを実装する. ここでは`OriginalModelSample.R5TorchNNSampleForHandyRL.R5TorchNNSampleForHandyRL`により深層学習モデルを構築する前提とする. 

コンフィグファイル`sample_config.yml`の`policy_config`->`Learner`->`model_config`を適宜編集して深層学習モデルを新たに定義することができる.

```Yaml
dense:
    layers:
        - ["Linear",{"out_features": 128}]
        - ["ReLU",{}]
        - ["ResidualBlock",{
            "layers":[
                ["Linear",{"out_features": 128}],
                ["BatchNorm1d",{}]
            ]}]
        - ["ReLU",{}]
        - ["ResidualBlock",{
            "layers":[
                ["Linear",{"out_features": 128}],
                ["BatchNorm1d",{}]
            ]}]

```

`layers`の中で層を増やしたりノード数を増やしたりして深層学習モデルの構造を新たに定義する. 前に設定した`observation_space`や`action_space`によって入力層や出力層が決まる. 

対戦を実行する際は強化学習フレームワークから独立させてPolicyを使用するためのインターフェースによりpolicyを作成する(ここでは提供されている`ASRCAISim1.addons.HandyRLUtility.StandaloneHandyRLPolicy.StandaloneHandyRLPolicy`を使用). importして呼べるようにしておく.

強化学習を前提としない場合は投稿プログラム内で適当な値を返すダミーpolicyを実装しておく.

In [ ]:
import os, yaml
from OriginalModelSample.R5TorchNNSampleForHandyRL import R5TorchNNSampleForHandyRL
from ASRCAISim1.addons.HandyRLUtility.distribution import getActionDistributionClass
from ASRCAISim1.addons.HandyRLUtility.StandaloneHandyRLPolicy import StandaloneHandyRLPolicy

model_config=yaml.safe_load(open(os.path.join(os.path.dirname(__file__),"model_config.yaml"),"r"))
weightPath = None
isDeterministic=False #決定論的に行動させたい場合はTrue、確率論的に行動させたい場合はFalseとする
policy = StandaloneHandyRLPolicy(R5TorchNNSampleForHandyRL,model_config,weightPath,getActionDistributionClass,isDeterministic)

## Factoryへの追加

配布した`make_agent_rl/make_agent.ipynb`に従う.

### 独自のAgentの登録

学習を実行するときに作成したエージェントを呼べるように登録しておく必要がある.

`./sample_config.yml`の"env_args"->"env"の値で実際に環境構築を行うモジュールを選択している(デフォルトでは`sample`という名前になっていて, `handyrl.envs.SampleEnv.sample.py`が利用されることになる.)が, シミュレーション環境構築時(`handyrl.envs.SampleEnv.sample.py`のL56-76の部分)でエージェントの登録を行っている.

```Python
# エージェントの登録
userModelID=args["userModelID"]
userModuleID=args["userModuleID"]
with open(os.path.join(userModuleID, args["modelargs"])) as f:
    model_args = json.load(f)

module = importlib.import_module(userModuleID)
assert hasattr(module, "getUserAgentClass")
assert hasattr(module, "getUserAgentModelConfig")
assert hasattr(module, "isUserAgentSingleAsset")
assert hasattr(module, "getUserPolicy")

agentClass = module.getUserAgentClass(model_args)
addPythonClass("Agent", "Agent_"+userModelID, agentClass)
Factory.addDefaultModel(
    "Agent",
    "Agent_"+userModelID,
    {
        "class": "Agent_"+userModelID,
        "config": module.getUserAgentModelConfig(model_args)
    }
)
```

`userModuleID`という名前のディレクトリに`__init__.py`があって, インポートしている. デフォルトでは`Test`という名前としている(`./Test`以下を実際に確認されたい.).

`./sample_config.yml`の"env_args"の"userModelID"の値`userModelID`が`Agent_{userModelID}`という形でクラス名として登録される. この名前と`R5_contest_learning_config_{M,S}.json`の"AgentConfigDispatcher"に記載の"Learner_e"の"model"の値を一致させることで作成したエージェントによる学習が実行可能となる(デフォルトでは"userModelID"は`Sample`としているので`Agent_Sample`としている. 各自確認されたい.).

ここでは特に扱っていないが, 並列実行する場合は全てのインスタンス上でFactoryへの追加が行われる必要がある(EnvironmentはWorkerごとにインスタンス化される(__init__がWorkerごとに呼ばれる)ため, 全てのインスタンスでFactoryへのモデル追加が独立に行われる).

デフォルトでは中央集権型として扱うため`R5_contest_learning_config_M.json`を自作エージェントに合わせるように編集している.

エージェント登録の別の方法として, OriginalModelSampleに独自のクラスを作成しておいて(ビルドして`site-packages`を更新)`configs/R5_contest_agent_ruler_reward_models.json`で"Factory"の"Agent"項目で好きな名前(`新エージェント名`とする)を登録して, その"class"において, 作成した独自のクラス名を記載しておくことも可能. その際は"AgentConfigDispatcher"に記載の"Learner_e"の"model"の値を`新エージェント名`にする.

投稿可能なプログラムを作成するときはクラスを記述する部分を別のpyファイルとして作成してインポートできるようにしたり, `__init__.py`に直接書き込んでその中で呼べるようにしておく必要がある. 以下は`__init__.py`に直接書き込む場合の例.

```Python
def getUserAgentClass(args={}):
    import 独自のクラス名
    return 独自のクラス名

class 独自のクラス名():
    ...
```

### 独自のRewardの登録


シミュレーション環境構築時(`handyrl.envs.SampleEnv.sample.py`のL79-95の部分)で報酬の登録を行っている.

```Python
# 報酬の登録
userRewardModuleID = args["userRewardModuleID"]
with open(args["rewardConfig"]) as f:
    reward_config = json.load(f)

reward_module = importlib.import_module(userRewardModuleID)

rewardClass = reward_module.MyReward
addPythonClass("Reward", userRewardModuleID, rewardClass)
Factory.addDefaultModel(
    "Reward",
    userRewardModuleID,
    {
        "class": "MyReward",
        "config": reward_config
    }
)
```

`seminar/MyReward.py`の`MyReward`モジュールを読み込んで`seminar/reward_config.json`で記述されている設定ファイルを渡して登録をしている. この実装ではクラス名は`MyReward`で固定する必要がある. `seminar/MyReward.py`では, `onInnerStepEnd`メソッドにおいてインナーステップ終了時に残存機数に応じて報酬を与えるシンプルなものとなっている. 適宜改修されたい. また, `docs/基準シミュレータ 取扱説明書.pdf`の5.3項も参照されたい. `OriginalModelSample`にも`R5PyRewardSample01.py`などに報酬の実装例があるため, こちらも参照すること.

またエージェントの場合と同様に, OriginalModelSampleに独自のクラスを作成しておいて(ビルドして`site-packages`を更新)`configs/R5_contest_agent_ruler_reward_models.json`で"Factory"の"Reward"項目で好きな名前(`新報酬名`とする)を登録して, その"class"において, 作成した独自のクラス名を記載しておくことも可能.

学習設定ファイル`configs/R5_contest_learning_config_{M/S}.json`の"Reward"で登録する報酬を以下のようにリストで渡すとその合計値を実際の報酬として返す. ここで実装している`seminar/MyReward.py`を報酬として学習させたい場合, `sample_config.yml`の"env_args"の"userRewardModuleID"の値を以下のように"model"の値として記述しておく. "target"を以下のように"All"とすると場に存在する者すべての陣営及びAgentが計算対象となる. 詳細は`docs/基準シミュレータ 取扱説明書.pdf`の表4.6-2などを参照.

```json
[
    {"model":"MyReward","target":"All"},
    {"model":"MyWinLoseReward","target":"All"}
]
```

以下は`MyReward.py`の実装例.

In [ ]:
from ASRCAISim1.libCore import TeamReward, nljson, Fighter


class MyReward(TeamReward):
    """
    チーム全体で共有する報酬は TeamReward を継承し、
    個別の Agent に与える報酬は AgentReward を継承する。
    """
    def __init__(self, modelConfig: nljson, instanceConfig: nljson):
        super().__init__(modelConfig, instanceConfig)
        if(self.isDummy):
            return #Factory によるダミー生成のために空引数でのインスタンス化に対応させる


    def onEpisodeBegin(self):
        """
        エピソード開始時の処理(必要に応じてオーバーライド)
        基底クラスにおいて config に基づき報酬計算対象の設定等が行われるため、
        それ以外の追加処理や設定の上書きを行いたい場合のみオーバーライドする。
        """
        super().onEpisodeBegin()


    def onStepBegin(self):
        """
        step 開始時の処理(必要に応じてオーバーライド)
        基底クラスにおいて reward(step 報酬)を 0 にリセットしているため、
        オーバーライドする場合、基底クラスの処理を呼び出すか、同等の処理が必要。
        """
        super().onEpisodeBegin()
    
    
    def onInnerStepBegin(self):
        """
        インナーステップ開始時の処理(必要に応じてオーバーライド)
        デフォルトでは何も行わないが、より細かい報酬計算が必要な場合に使用可能。
        """
        pass
    
    
    def onInnerStepEnd(self):
        """
        インナーステップ終了時の処理(必要に応じてオーバーライド)
        一定周期で呼び出されるため、極力この関数で計算する方が望ましい。
        """
        for team in self.reward: # team に属している Asset(Fighter)を取得する例
            for f in self.manager.getAssets(lambda a:a.getTeam()==team and isinstance(a,Fighter)):
                if(f().isAlive()):
                    self.reward[team] += 0.1 #例えば、残存数に応じて報酬を与える場合



## 学習の実行

作成したエージェントを学習する. 学習に使用する環境やモデルや学習条件などが`./sample_config.yml`として与えられている. 特に"env_args"の"userModuleID"と"model_args"にはそれぞれ実装したエージェント(`MyAgent.py`)が保存されるディレクトリ(デフォルトでは`Test`)とその引数ファイル(`args.json`)を設定する(`./sample_config.yml`参照). 


エージェントに対する設定ファイル(`agent_config.json`)において, 今回はobservationは数値ベクトル(`numpy.ndarray`)としているので以下の設定とする.

- use_image_observation: false
- use_vector_observation: true


例えば学習条件としてエポック数を変えたい場合は`sample_config.yml`の`train_args`で`epochs`を変えればよい(-1に設定すると上限なしとなる.).

```Yaml
train_args:
    epochs: 5
```

詳細は`docs/基準シミュレータ 取扱説明書.pdf`の6.7.4を参照すること.

エージェント(`MyAgent.py`)とその引数ファイル(`args.json`), エージェントに対する設定ファイル(`agent_config.json`)を`./Test`以下に格納しておく. すると`./Test`は以下のようなディレクトリ構造になる.

```bash
Test
├─ __init__.py
├─ agent_config.json
├─ args.json
└─ MyAgent.py
```

そして以下のコマンドを実行する.

In [ ]:
%cd /content/seminar
!python main.py sample_config.yml --train

学習済みモデルは`./results/Multi/YYYYmmddHHMMSS/policies/checkpoints`以下に保存される(pthファイル).

## 投稿可能なプログラム一式としてまとめる

学習済みモデル(例えば`Learner-latest.pth`)を`./Test`以下に格納する. `args.json`の"weightPath"の値を`Learner-latest.pth`としておく. また, 学習時に使用したモデルの設定ファイルと同等のものとして`model_config.yaml`を格納する(`sample_config.yml`の"policy_config"などの内容と整合することを確認しておく.). そして最終的に以下のようなディレクトリ構造となることを確認.

```bash
Test
├─ __init__.py
├─ agent_config.json
├─ args.json
├─ Learner-latest.pth
├─ model_config.yaml
└─ MyAgent.py
```

## 対戦を実行する

作成したエージェントを初期行動判断モデルと戦わせる. 事前に`/content/simulator_dist`直下に上記で作成したプログラム一式を配置しておく.

In [ ]:
%cd /content/simulator_dist
!python validate.py --num-validation 1 --agent-module-path ./Test

適宜`--color`を"Blue"や"Red"に変えて陣営の種類に応じた行動が取れているかなどを確認する(実際の対戦では陣営の色はランダムに決まる). また, `--movie`と`--visualize`を`1`にして実際にどのように動いているか(google colaboratory上ではできない)を確認するなりログを参考にしてobservationの作成方法に工夫を施したり`sample_config.yml`で使用するモデルを変えたり学習の仕方に工夫を施すなどしてアルゴリズムをよりよくする.

## 応募用ファイルを作成する

作成したプログラムをzipファイルとして圧縮する.

In [ ]:
%cd /content/simulator_dist
!zip -r submit ./Test